# Objective:

To analyze and compare the chemical properties of ten randomly selected molecules using three different computational approaches:

1. **RDKit** – local cheminformatics analysis directly from molecular structures.
2. **PubChemPy** – property retrieval via the PubChem Python interface.
3. **PubChem API with BeautifulSoup** – data extraction from the PubChem website through web scraping.

The goal is to evaluate the consistency, accessibility, and scope of chemical property information obtained through these three methods.

These chemical properties are LogP, molecular weight, number of rotatable bonds and number of heavy atoms, which are molecular descriptors — quantitative features that summarize a molecule’s physical and chemical characteristics. In drug design, they’re crucial because they correlate with pharmacokinetics (absorption, distribution, metabolism, excretion — ADME) and drug-likeness.


| Descriptor |	What it measures |	importance |
| -----------|-------------------|-------------|
| LogP | Lipophilicity (balance between hydrophobic and hydrophilic behavior) |	Affects membrane permeability, solubility, bioavailability. Too high → insoluble and toxic; too low → poor cell penetration.	
Molecular Weight (MW) |	Total mass of the molecule |	Influences diffusion and transport. High MW compounds often have poor absorption and oral bioavailability.
Number of Rotatable Bonds |	Molecular flexibility |	More flexibility → higher entropic cost for binding → weaker binding and worse oral bioavailability. Fewer rotatable bonds → better conformational stability.	
Number of Heavy Atoms |	Count of non-hydrogen atoms |	Reflects molecular size and complexity; correlated with MW and surface area.


In drug design, these fall under Lipinski’s Rule of Five and similar drug-likeness filters:
- MW ≤ 500
- LogP ≤ 5
- ≤ 5 hydrogen bond donors
- ≤ 10 hydrogen bond acceptors
- ≤ 10 rotatable bonds



---

Library requirements:

- `PubchemPy`
- `RDKit`
- `BeautifulSoup`

# Install the libraries

In [ ]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import PandasTools
from rdkit.Chem.ChemUtils import SDFToCSV
import pubchempy as pcp
import numpy as np
import random
import glob
import seaborn as sns
import matplotlib

# Fetch 10 random compounds from Pubchem

In [ ]:
PATH = '/input/'
# choose 10 random PubChem CIDs
random_cids = random.sample(range(1, 100000), 10)

# Retrieve the specified compound records from PubChem.
compounds = pcp.get_compounds(random_cids, 'cid')

# convert to DataFrame with standard PubChem fields
df = pd.DataFrame([c.to_dict() for c in compounds])

# save to CSV
df.to_csv(PATH + 'ten_random_chemicals.csv', index=False)

# optional: convert to html
df.to_html(PATH + 'ten_random_chemicals.html')


# Read the dataset

In [34]:
df = pd.read_csv(PATH + 'ten_random_chemicals.csv', sep=',')

df.head()

,cid,elements,atoms,bonds,coordinate_type,charge,molecular_formula,molecular_weight,connectivity_smiles,smiles,...,multipoles_3d,conformer_rmsd_3d,effective_rotor_count_3d,pharmacophore_features_3d,mmff94_partial_charges_3d,mmff94_energy_3d,conformer_id_3d,shape_selfoverlap_3d,feature_selfoverlap_3d,shape_fingerprint_3d
0,5075,"['O', 'O', 'O', 'N', 'C', 'C', 'C', 'C', 'C', ...","[{'aid': 1, 'number': 8, 'element': 'O', 'y': ...","[{'aid1': 1, 'aid2': 7, 'order': 1, 'style': 3...",2d,0,C17H21NO3,287.35,CC(CC(C1=CC=C(C=C1)O)O)NCC2=CC=C(C=C2)O,CC(CC(C1=CC=C(C=C1)O)O)NCC2=CC=C(C=C2)O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56103,"['O', 'N', 'N', 'N', 'N', 'C', 'C', 'C', 'C', ...","[{'aid': 1, 'number': 8, 'element': 'O', 'y': ...","[{'aid1': 1, 'aid2': 11, 'order': 2}, {'aid1':...",2d,0,C24H18N4O,378.40,CC1=NC(=CC2=CC=CC=C2)C(=O)N1C3=CC=CC(=C3)C4=NC...,CC1=NC(=CC2=CC=CC=C2)C(=O)N1C3=CC=CC(=C3)C4=NC...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80479,"['O', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","[{'aid': 1, 'number': 8, 'element': 'O', 'y': ...","[{'aid1': 1, 'aid2': 7, 'order': 1}, {'aid1': ...",2d,0,C10H9NO,159.18,C1=CC=C2C(=C1)C(=CC=N2)CO,C1=CC=C2C(=C1)C(=CC=N2)CO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37850,"['Cl', 'S', 'N', 'C', 'C', 'C', 'C', 'C', 'H',...","[{'aid': 1, 'number': 17, 'element': 'Cl', 'y'...","[{'aid1': 2, 'aid2': 8, 'order': 1}, {'aid1': ...",2d,0,C5H14ClNS,155.69,C[N+](C)(C)CCS.[Cl-],C[N+](C)(C)CCS.[Cl-],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,88876,"['N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","[{'aid': 1, 'number': 7, 'element': 'N', 'y': ...","[{'aid1': 1, 'aid2': 3, 'order': 2, 'style': 8...",2d,1,C8H10N+,120.17,C[N+]1=CC=C(C=C1)C=C,C[N+]1=CC=C(C=C1)C=C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df.columns

Index(['cid', 'elements', 'atoms', 'bonds', 'coordinate_type', 'charge',
       'molecular_formula', 'molecular_weight', 'connectivity_smiles',
       'smiles', 'inchi', 'inchikey', 'iupac_name', 'xlogp', 'exact_mass',
       'monoisotopic_mass', 'tpsa', 'complexity', 'h_bond_donor_count',
       'h_bond_acceptor_count', 'rotatable_bond_count', 'fingerprint',
       'cactvs_fingerprint', 'heavy_atom_count', 'isotope_atom_count',
       'atom_stereo_count', 'defined_atom_stereo_count',
       'undefined_atom_stereo_count', 'bond_stereo_count',
       'defined_bond_stereo_count', 'undefined_bond_stereo_count',
       'covalent_unit_count', 'volume_3d', 'multipoles_3d',
       'conformer_rmsd_3d', 'effective_rotor_count_3d',
       'pharmacophore_features_3d', 'mmff94_partial_charges_3d',
       'mmff94_energy_3d', 'conformer_id_3d', 'shape_selfoverlap_3d',
       'feature_selfoverlap_3d', 'shape_fingerprint_3d'],
      dtype='object')

# RDkit-based calculation of molecular properties

## Calculate MW, LogP, Heavy Atom Numbers and Rotatable bonds

In [36]:
prop_from_rdkit = pd.DataFrame({'SMILES': df['smiles'], 'CID': df['cid']})

# MW Calculation
prop_from_rdkit ['MW'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.MolWt(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# LogP Calculation
prop_from_rdkit ['LogP'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.MolLogP(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# Heavy Atom Count Calculation
prop_from_rdkit ['Heavy Atom Count'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.HeavyAtomCount(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# Num. Rotatable Bonds Calculation
prop_from_rdkit ['Num. Rotatable Bonds'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.NumRotatableBonds(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

prop_from_rdkit

,SMILES,CID,MW,LogP,Heavy Atom Count,Num. Rotatable Bonds
0,CC(CC(C1=CC=C(C=C1)O)O)NCC2=CC=C(C=C2)O,5075,287.359,2.6996,21,6
1,CC1=NC(=CC2=CC=CC=C2)C(=O)N1C3=CC=CC(=C3)C4=NC...,56103,378.435,5.0360,29,3
2,C1=CC=C2C(=C1)C(=CC=N2)CO,80479,159.188,1.7271,12,1
3,C[N+](C)(C)CCS.[Cl-],37850,155.694,-2.3736,8,2
4,C[N+]1=CC=C(C=C1)C=C,88876,120.175,1.1541,9,1
5,C[N+](C)(C)C(=O)OCC[C@H]1C[C@H]1CCOC(=O)[N+](C...,50670,302.415,2.0883,21,6
6,CC(C)CP(=S)(CC(C)C)[S-].[Na+],83375,232.330,0.2438,12,4
7,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,71738,512.560,7.8038,29,6
8,C1=CC2=C(C(=C1)N)C(=O)C3=C(C2=O)C(=CC=C3)Cl,8327,257.676,2.6976,18,0
9,CN(C)P(=O)(N1CC1)N(C)C,96669,177.188,0.5333,11,3


# PubchemPy-based calculation of molecular properties

## Calculate MW, LogP, Heavy Atom Numbers and Rotatable bonds:

In [37]:
# Add the CID Column
prop_from_pubchempy = pd.DataFrame()
prop_from_pubchempy ['CID'] = df['cid']

# MW Calculation
prop_from_pubchempy['MW'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.MolWt(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# LogP Calculation
prop_from_pubchempy ['LogP'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.MolLogP(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# Heavy Atom Count Calculation
prop_from_pubchempy ['Heavy Atom Count'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.HeavyAtomCount(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# Num. Rotatable Bonds Calculation
prop_from_pubchempy ['Num. Rotatable Bonds'] = df['smiles'].apply(lambda smiles: Chem.Descriptors.NumRotatableBonds(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

prop_from_pubchempy

,CID,MW,LogP,Heavy Atom Count,Num. Rotatable Bonds
0,5075,287.359,2.6996,21,6
1,56103,378.435,5.0360,29,3
2,80479,159.188,1.7271,12,1
3,37850,155.694,-2.3736,8,2
4,88876,120.175,1.1541,9,1
5,50670,302.415,2.0883,21,6
6,83375,232.330,0.2438,12,4
7,71738,512.560,7.8038,29,6
8,8327,257.676,2.6976,18,0
9,96669,177.188,0.5333,11,3


# Pubchem API-based calculation of molecular properties

## SMILES from PubChem API:
*with CID numbers*

In [38]:
def get_smiles_from_api(cids):
  try:
    Url = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cids}/xml')
    soup = bs(Url.content, 'xml')
    #soup.raise_for_status() # Error Handling

    smiles = soup.find(name = 'PC-Urn_label', string = 'SMILES')
    smiles_parent = smiles.find_parent('PC-InfoData')
    smiles_string = smiles_parent.find('PC-InfoData_value_sval').string
    return smiles_string
  except requests.exceptions.RequestException as e: # Error Handling
    print(f"Error fetching data for '{cids}': {e}")
    return None


smiles_from_api = [get_smiles_from_api(cids) for cids in df ['cid']]
smiles_from_api

['CC(CC(C1=CC=C(C=C1)O)O)NCC2=CC=C(C=C2)O',
 'CC1=NC(=CC2=CC=CC=C2)C(=O)N1C3=CC=CC(=C3)C4=NC5=CC=CC=C5N4',
 'C1=CC=C2C(=C1)C(=CC=N2)CO',
 'C[N+](C)(C)CCS.[Cl-]',
 'C[N+]1=CC=C(C=C1)C=C',
 'C[N+](C)(C)C(=O)OCC[C@H]1C[C@H]1CCOC(=O)[N+](C)(C)C',
 'CC(C)CP(=S)(CC(C)C)[S-].[Na+]',
 'C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2C[C@H](C4=C3CC[C@@H](C4)O)CI)C',
 'C1=CC2=C(C(=C1)N)C(=O)C3=C(C2=O)C(=CC=C3)Cl',
 'CN(C)P(=O)(N1CC1)N(C)C']

## Calculate MW, LogP, Heavy Atom Numbers and Rotatable bonds:

In [39]:
# Make a DataFrame from smiles_from_api list
prop_from_api = pd.DataFrame()
prop_from_api ['SMILES'] = smiles_from_api

# Add the CID Column
prop_from_api ['CID'] = df['cid']

# MW Calculation
prop_from_api ['MW'] = prop_from_api ['SMILES'].apply(lambda smiles: Chem.Descriptors.MolWt(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# LogP Calculation
prop_from_api ['LogP'] = prop_from_api ['SMILES'].apply(lambda smiles: Chem.Descriptors.MolLogP(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# Heavy Atom Count Calculation
prop_from_api ['Heavy Atom Count'] = prop_from_api ['SMILES'].apply(lambda smiles: Chem.Descriptors.HeavyAtomCount(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

# Num. Rotatable Bonds Calculation
prop_from_api ['Num. Rotatable Bonds'] = prop_from_api ['SMILES'].apply(lambda smiles: Chem.Descriptors.NumRotatableBonds(Chem.MolFromSmiles(smiles))
if Chem.MolFromSmiles(smiles) else None)

prop_from_api

,SMILES,CID,MW,LogP,Heavy Atom Count,Num. Rotatable Bonds
0,CC(CC(C1=CC=C(C=C1)O)O)NCC2=CC=C(C=C2)O,5075,287.359,2.6996,21,6
1,CC1=NC(=CC2=CC=CC=C2)C(=O)N1C3=CC=CC(=C3)C4=NC...,56103,378.435,5.0360,29,3
2,C1=CC=C2C(=C1)C(=CC=N2)CO,80479,159.188,1.7271,12,1
3,C[N+](C)(C)CCS.[Cl-],37850,155.694,-2.3736,8,2
4,C[N+]1=CC=C(C=C1)C=C,88876,120.175,1.1541,9,1
5,C[N+](C)(C)C(=O)OCC[C@H]1C[C@H]1CCOC(=O)[N+](C...,50670,302.415,2.0883,21,6
6,CC(C)CP(=S)(CC(C)C)[S-].[Na+],83375,232.330,0.2438,12,4
7,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,71738,512.560,7.8038,29,6
8,C1=CC2=C(C(=C1)N)C(=O)C3=C(C2=O)C(=CC=C3)Cl,8327,257.676,2.6976,18,0
9,CN(C)P(=O)(N1CC1)N(C)C,96669,177.188,0.5333,11,3


# Compare the Calculated Chemical Properties (*Molecular Weight, LogP, Number of Rotatable Bonds and Number of Heavy Atoms*) using 3 Methods:
1. RDkit
2. PubChemPy
3. PubChem API

In [42]:
# Make a comparison DataFrame
comparison_df = pd.DataFrame()

# Add the actual values of MW, LogP, Heavy Atom Count and Num. Rotatable Bonds to the comparison dataframe
comparison_df ['MW_RDkit'] = round(prop_from_rdkit['MW'], 3)
comparison_df ['MW_PubChemPy'] = round(prop_from_pubchempy['MW'], 3)
comparison_df ['MW_API'] = round(prop_from_api ['MW'], 3)

comparison_df ['LogP_RDkit'] = round(prop_from_rdkit['LogP'], 3)
comparison_df ['LogP_PubChemPy'] = round(prop_from_pubchempy['LogP'], 3)
comparison_df ['LogP_API'] = round(prop_from_api ['LogP'], 3)

comparison_df ['Heavy Atom Count_RDkit'] = prop_from_rdkit['Heavy Atom Count']
comparison_df ['Heavy Atom Count_PubChemPy'] = prop_from_pubchempy['Heavy Atom Count']
comparison_df ['Heavy Atom Count_API'] = prop_from_api ['Heavy Atom Count']

comparison_df ['Num. Rotatable Bonds_RDkit'] = prop_from_rdkit['Num. Rotatable Bonds']
comparison_df ['Num. Rotatable Bonds_PubChemPy'] = prop_from_pubchempy['Num. Rotatable Bonds']
comparison_df ['Num. Rotatable Bonds_API'] = prop_from_api ['Num. Rotatable Bonds']

# Compare the MW in all 3 Methods
comparison_df ['MW_Comp.'] = np.where((prop_from_rdkit['MW'] == prop_from_pubchempy['MW']) &
 (prop_from_rdkit['MW'] == prop_from_api ['MW']) &
  (prop_from_pubchempy['MW'] == prop_from_api ['MW']), 'True', 'False' )

# # Compare the LogP in all 3 Methods
comparison_df ['LogP_Comp.'] = np.where((prop_from_rdkit['LogP'] == prop_from_pubchempy['LogP']) &
 (prop_from_rdkit['LogP'] == prop_from_api ['LogP']) &
  (prop_from_pubchempy['LogP'] == prop_from_api ['LogP']), 'True', 'False' )

# Compare the Heavy Atom Count in all 3 Methods
comparison_df ['Heavy Atom Count_Comp.'] = np.where((prop_from_rdkit['Heavy Atom Count'] == prop_from_pubchempy['Heavy Atom Count']) &
 (prop_from_rdkit['Heavy Atom Count'] == prop_from_api ['Heavy Atom Count']) &
  (prop_from_pubchempy['Heavy Atom Count'] == prop_from_api ['Heavy Atom Count']), 'True', 'False' )

# Compare Num. Rotatable Bonds in all 3 Methods
comparison_df ['Num. Rotatable Bonds_Comp.'] = np.where((prop_from_rdkit['Num. Rotatable Bonds'] == prop_from_pubchempy['Num. Rotatable Bonds']) &
 (prop_from_rdkit['Num. Rotatable Bonds'] == prop_from_api ['Num. Rotatable Bonds']) &
  (prop_from_pubchempy['Num. Rotatable Bonds'] == prop_from_api ['Num. Rotatable Bonds']), 'True', 'False' )

In [43]:
comparison_df

,MW_RDkit,MW_PubChemPy,MW_API,LogP_RDkit,LogP_PubChemPy,LogP_API,Heavy Atom Count_RDkit,Heavy Atom Count_PubChemPy,Heavy Atom Count_API,Num. Rotatable Bonds_RDkit,Num. Rotatable Bonds_PubChemPy,Num. Rotatable Bonds_API,MW_Comp.,LogP_Comp.,Heavy Atom Count_Comp.,Num. Rotatable Bonds_Comp.
0,287.359,287.359,287.359,2.700,2.700,2.700,21,21,21,6,6,6,True,True,True,True
1,378.435,378.435,378.435,5.036,5.036,5.036,29,29,29,3,3,3,True,True,True,True
2,159.188,159.188,159.188,1.727,1.727,1.727,12,12,12,1,1,1,True,True,True,True
3,155.694,155.694,155.694,-2.374,-2.374,-2.374,8,8,8,2,2,2,True,True,True,True
4,120.175,120.175,120.175,1.154,1.154,1.154,9,9,9,1,1,1,True,True,True,True
5,302.415,302.415,302.415,2.088,2.088,2.088,21,21,21,6,6,6,True,True,True,True
6,232.330,232.330,232.330,0.244,0.244,0.244,12,12,12,4,4,4,True,True,True,True
7,512.560,512.560,512.560,7.804,7.804,7.804,29,29,29,6,6,6,True,True,True,True
8,257.676,257.676,257.676,2.698,2.698,2.698,18,18,18,0,0,0,True,True,True,True
9,177.188,177.188,177.188,0.533,0.533,0.533,11,11,11,3,3,3,True,True,True,True


Therefore, based on the results of the comparison, the physiochemical and structural molecular descriptors measured by three different methods, namely RDkit, PubchemPy library and Pubchem API, were identical.

In [47]:
# Import seaborn library
import seaborn as sns

# Declaring the cm variable by the
# color palette from seaborn
cm = sns.color_palette("viridis", as_cmap=True)

# Visualizing the DataFrame with set precision
print("\nROUNDED DATAFRAME WITH COLOR STYLING:")
comparison_df.style.background_gradient(cmap=cm).format(precision = 3)



ROUNDED DATAFRAME WITH COLOR STYLING:


,MW_RDkit,MW_PubChemPy,MW_API,LogP_RDkit,LogP_PubChemPy,LogP_API,Heavy Atom Count_RDkit,Heavy Atom Count_PubChemPy,Heavy Atom Count_API,Num. Rotatable Bonds_RDkit,Num. Rotatable Bonds_PubChemPy,Num. Rotatable Bonds_API,MW_Comp.,LogP_Comp.,Heavy Atom Count_Comp.,Num. Rotatable Bonds_Comp.
0,287.359,287.359,287.359,2.700,2.700,2.700,21,21,21,6,6,6,True,True,True,True
1,378.435,378.435,378.435,5.036,5.036,5.036,29,29,29,3,3,3,True,True,True,True
2,159.188,159.188,159.188,1.727,1.727,1.727,12,12,12,1,1,1,True,True,True,True
3,155.694,155.694,155.694,-2.374,-2.374,-2.374,8,8,8,2,2,2,True,True,True,True
4,120.175,120.175,120.175,1.154,1.154,1.154,9,9,9,1,1,1,True,True,True,True
5,302.415,302.415,302.415,2.088,2.088,2.088,21,21,21,6,6,6,True,True,True,True
6,232.330,232.330,232.330,0.244,0.244,0.244,12,12,12,4,4,4,True,True,True,True
7,512.560,512.560,512.560,7.804,7.804,7.804,29,29,29,6,6,6,True,True,True,True
8,257.676,257.676,257.676,2.698,2.698,2.698,18,18,18,0,0,0,True,True,True,True
9,177.188,177.188,177.188,0.533,0.533,0.533,11,11,11,3,3,3,True,True,True,True
